In [1]:
import gzip
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [2]:
print('reading data...')
data = list(readGz('train.json.gz'))
print('done')

reading data...
done


In [4]:
#problem1
import itertools
import random
known_ub_pair = [(d['userID'],d['businessID']) for d in data]  #get known user,business pair from original 200,000 training data
known_user = [d['userID'] for d in data]
known_business = [d['businessID'] for d in data]
known_user = list(set(known_user))
known_business = list(set(known_business))



In [5]:
count = 0
count_total = 100000
while True:
    index_u = random.randint(0,len(known_user)-1)
    index_b = random.randint(0,len(known_business)-1)
    sample = (known_user[index_u],known_business[index_b])
    if sample not in known_ub_pair:
        validation.append(sample)
        count += 1
    if count == count_total:
        break


In [6]:
#calculate accuracy of the baseline model on the validation set
y=[1]*100000+[0]*100000
businessCount = defaultdict(int)
totalPurchases = 0
for i in training:
    user,business = i[0],i[1]
    businessCount[business] += 1
    totalPurchases += 1
mostPopular = [(businessCount[x],x) for x in businessCount]
mostPopular.sort()
mostPopular.reverse()
return1 = set() #contain the business ID for the top 50% purchase
count = 0
for ic,i in mostPopular:
    count += ic
    return1.add(i)
    if count>totalPurchases/2:break
prediction = []
for i in range(len(validation)):
    if validation[i][1] in return1:
        prediction.append(1)
    else:
        prediction.append(0)
correct = [(a == b) for a,b in zip(y,prediction)]
accuracy = sum(correct)/len(correct)
print('accuracy of the baseline model on the validation set is',accuracy*100,' % ')

accuracy of the baseline model on the validation set is 62.6075  % 


In [12]:
#problem 2
return2 = set() #contain the business ID for the top 58% 
count = 0
for ic,i in mostPopular:
    count += ic
    return2.add(i)
    if count>totalPurchases*5.8/10:break
prediction2 = []
for i in range(len(validation)):
    if validation[i][1] in return2:
        prediction2.append(1)
    else:
        prediction2.append(0)
correct = [(a == b) for a,b in zip(y,prediction2)]
accuracy = sum(correct)/len(correct)
print('accuracy of the baseline model on the validation set is',accuracy*100,' % ')

accuracy of the baseline model on the validation set is 62.924  % 


In [28]:
#problem 3
training=data[:int(len(data)/2)]
userCat = defaultdict(list)
for u in known_user:
    cat = [d['categories'] for d in training if d['userID'] == u]
    combine_cat = [j for i in cat for j in i]  
    for i in combine_cat:
        userCat[u].append(i)
#userCat contain userID:[category]

In [32]:
busiCat = defaultdict(list)
for b in known_business:
    cat = [d['categories'] for d in training if d['businessID'] == b]
    combine_cat = [j for i in cat for j in i]
    for i in combine_cat:
        busiCat[b].append(i)

In [37]:
predictions = open("visit_prediction_p4.txt", 'w')
for l in open("pairs_Visit.txt"):
    if l.startswith("userID"):
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    if i not in known_business:
        predictions.write(u + '-' + i + ',' + '0\n')
    elif u in known_user:
        predictions.write(u + '-' + i + ',' + '1\n')
    else:
        if i in return2:
            predictions.write(u + '-' + i + ',' + '1\n')
        else:
            predictions.write(u + '-' + i + ',' + '0\n')
predictions.close()